In [ ]:
import pandas as pd
from sklearn.metrics import classification_report
import os
import warnings
from configs.data import MACHINE_LEARNING_DATASET_PATH
from machine_learning.utils import split_data, scale_dataset
from machine_learning.neural_networks.utils import get_tensorflow_version, plot_history
from machine_learning.neural_networks.deep_fnn import train_deep_fnn_model
from tensorflow.keras.models import load_model
from configs.enums import Column, RiskClassifications
import numpy as np
import matplotlib.pyplot as plt
from configs.data import MERGED_DATASET_PATH, OUT_PATH, MODELS_PATH
import shap

warnings.simplefilter(action='ignore', category=FutureWarning)
np.random.seed(0)

In [ ]:
def tune_ann_model(df, units, dropout_rates, learning_rates):
    least_val_loss = float('inf')
    least_val_loss_params = []
    
    i = 1 
    max = len(units) * len(dropout_rates) * len(learning_rates)
    for u in units:
        for dr in dropout_rates:
            for lr in learning_rates:  
                print(f"[{i}/{max}] Units: {u}; Dropout rate: {dr}; Learning rate: {lr};")
                
                model, _ = train_ann_model(
                    df, 
                    epochs=100, 
                    patience=20, 
                    units=u,
                    dropout_rate=dr,
                    learning_rate=lr,
                    verbose=0,
                    disable_save=True,
                    disable_plot_history=True,
                    disable_print_report=True)
                
                val_loss, val_acc = model.evaluate(x_test, test_labels)
                print(f"Loss: {val_loss}; Accuracy: {val_acc};")
                if val_loss < least_val_loss:
                    model.save(os.path.join(os.environ["OUTPUT_PATH"], "Risk_factor_ann_model.keras"))
                    least_val_loss = val_loss
                    least_val_loss_params = [u, dr, lr]
                    
                i += 1
                        
    print(least_val_loss_params)
    print(least_val_loss)

In [ ]:
tune_ann_model(df=df,
                units=[320],
                dropout_rates=[0.2],
                learning_rates=[0.002])

In [ ]:
model, _ = train_ann_model(
                        df, 
                        epochs=100, 
                        patience=10, 
                        units=320,
                        dropout_rate=0.2,
                        learning_rate=0.002,
                        verbose=2,
                        disable_save=True)

val_loss, val_acc = model.evaluate(x_test, test_labels)
print(f"Loss: {val_loss}; Accuracy: {val_acc};")

In [ ]:
y_pred_train = model.predict(x_train).argmax(axis=1)
print(classification_report(train_labels, y_pred_train))

y_pred_valid = model.predict(x_val).argmax(axis=1)
print(classification_report(val_labels, y_pred_valid))

y_pred = model.predict(x_test).argmax(axis=1)
print(classification_report(test_labels, y_pred))